In [1]:
#import necessary libraries
import warnings
import numpy as np
import pandas as pd
import math
warnings.simplefilter(action='ignore', category=FutureWarning) #suppress NumPy's future warning about scalars
print('x' in np.arange(5))

False


In [2]:
#determine if given s can be parsed as a float
def isNumber(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [3]:
#return only rows whose index is divisible by the given interval
def createSubset(data,interval):
    ind = [i for i in range(len(data)) if i%interval==0]
    #ind = [i for i in range(100) if i%10==0]
    return data.iloc[ind]

In [4]:
#clean data
def clean(data, errorCodes):
    newData = pd.DataFrame(columns=data.columns.values.tolist())
    for column in data:
        n = []
        for i, item in enumerate(data[column]):
            if item in errorCodes:
                n.append(float('nan'))
            else:
                n.append(item)
        newData[column]=n
    return newData

In [5]:
#import data from file
def parsefile(filename):
    return pd.read_csv(filename, sep=',', header=0)

#translate an array into a comma delimited string
def tocsvlist(x):
    output = ""
    for item in x:
        output = output + str(item) + ","
    return output[:-1] #skip the final ","

#take a csv file as input and output an arff
def dataTranformation(location, name, file, interval, errorCodes):
    #clean the data, remove empty values, 
    data = createSubset(clean(parsefile(file), errorCodes), interval)
    #remove NAN values from dataset, replace with arff code for missing value "?"
    data = data.fillna('?')
    colHeaders = data.columns.values.tolist()
    #if any value contains spaces, put the value in double quotes
    for m in range(len(colHeaders)):
        if colHeaders[m].find(" ") != -1:
            colHeaders[m] = '"' + colHeaders[m] + '"'
    #generate lists of each unique value for each column (possible attribute values, exclude ? from lists)
    #except numeric ones
    a = []
    for column in data:
        a.append(data[column].unique())
    for i in range(len(a)):
        a[i] = np.delete(a[i], np.where(a[i] == '?'))
        for j in range(len(a[i])):
            a[i][j] = str(a[i][j]).strip()
            if str(a[i][j]).find(" ") != -1:
                a[i][j] = '"' + a[i][j] + '"'
        a[i] = np.unique(a[i])
    #build a header for the arff
    header = "@relation " + name + "\n\n"
    i=0;
    #print a list of all possible values for each attribute (column). print "real" if the column is numeric
    for item in colHeaders:
        header = header + "@attribute " + item
        if len(a[i])>0 and isNumber(a[i][0]):
            header = header + " real\n"
        else:
            header = header + " {" + tocsvlist(a[i]) + "}\n"
        i=i+1
    #print the body of the data
    #print each row as an arff instance
    body = "\n\n@data\n"
    for row in data.itertuples():
        line = ""
        for i in range(len(colHeaders)):
            t = str(row[i+1]).strip()
            if t.find(" ") != -1:
                t = '"' + t + '"'
            line = line + t + ","
        line = line[:-1]
        body = body + line + "\n"
    #write string to a new file
    newfile = header + body
    f = open(location + ".arff", "w+")
    f.write(newfile)


In [6]:
#data transformation function takes arguments:
    #NameOfNewFile (without file extension)
    #NameOfNewRelation for new ARFF
    #Path\To\NameOfExistingCSVFile.csv
    #Interval as integer, if you want your arff file to contain a subset of data from the CSV file
    #errorCodes - array of anything you would like to remove from the dataset
dataTranformation('NameOfNewFile','NameOfNewRelation','Path\To\NameOfExistingCSVFile.csv', 1, ['NAN'])